In [1]:
import tensorflow as tf
import numpy as np
import input_data
mnist_width = 28
n_visible = mnist_width * mnist_width
n_hidden = 500
corruption_level = 0.3

In [3]:
# node for input data
X = tf.placeholder("float", [None, n_visible], name='X')
# node for corruption mask
mask = tf.placeholder("float", [None, n_visible], name='mask')
# ndoe for hidden varables
W_init_max = 4 * np.sqrt(6. / (n_visible + n_hidden))
W_init = tf.random_uniform(shape=[n_visible, n_hidden],
                          minval=-W_init_max,
                          maxval=W_init_max)
W = tf.Variable(W_init, name='W')
b = tf.Variable(tf.zeros([n_hidden]), name='b')

W_prime = tf.transpose(W)
b_prime = tf.Variable(tf.zeros([n_visible]), name='b_prime')



In [10]:
def model(X,mast,W,b,W_prime,b_prime):
    tilde_X = mask * X
    Y = tf.nn.sigmoid(tf.matmul(tilde_X, W) + b)
    Z = tf.nn.sigmoid(tf.matmul(Y, W_prime) + b_prime)
    return Z

In [11]:
#create cost function
Z = model(X, mask, W, b, W_prime, b_prime)
cost = tf.reduce_sum(tf.pow(X - Z, 2))
train_op = tf.train.GradientDescentOptimizer(0.02).minimize(cost)

In [15]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [21]:
with tf.Session() as sess:
    tf.initialize_all_variables().run()
    
    for i in range(100):
        for start, end in zip(range(0, len(trX), 128), range(128, len(trX), 128)):
            input_ = trX[start:end]
            mask_np = np.random.binomial(1,1-corruption_level, input_.shape)
            sess.run(train_op, feed_dict={X: input_, mask: mask_np})
        
        mask_np = np.random.binomial(1,1-corruption_level,teX.shape)
                              
        print(i, sess.run(cost, feed_dict={X: teX, mask: mask_np}))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
(0, 113883.03)
(1, 93137.93)
(2, 86729.414)
(3, 82193.414)
(4, 78145.969)
(5, 76958.773)
(6, 72465.375)
(7, 71505.008)
(8, 69961.609)
(9, 69190.5)
(10, 69315.062)
(11, 68507.156)
(12, 68148.219)
(13, 66382.594)
(14, 65061.297)
(15, 64461.16)
(16, 64595.922)
(17, 64348.312)
(18, 63774.156)
(19, 64445.891)
(20, 62591.105)
(21, 63401.18)
(22, 61741.25)
(23, 62288.738)
(24, 61591.004)
(25, 61480.23)
(26, 61946.855)
(27, 59919.984)
(28, 60924.688)
(29, 61571.391)
(30, 60883.949)
(31, 60477.176)
(32, 60193.508)
(33, 61132.371)
(34, 61147.797)
(35, 59913.059)
(36, 60063.188)
(37, 60022.25)
(38, 60007.148)
(39, 59828.496)
(40, 61757.469)
(41, 59614.172)
(42, 59178.797)
(43, 60815.516)


KeyboardInterrupt: 